# Текстовое ранжирование с помощью библиотеки Whoosh

В этом примере мы рассмотрим как с помощью библиотеки whoosh можно:
- проиндексировать корпус текстовых документов
- организовать по ней поиск, в котором будет использоваться ранжирование по формуле BM25
- посчитать метрики качества такого поиска

Будем использовать датасет <a href="https://microsoft.github.io/msmarco/">MS MARCO</a>.

Этот датасет сам по себе очень большой и состоит из нескольких частей, поэтому, для удобства, мы сделали из него маленький сэмпл,
который надо будет скачать и положить в папку data в корне проекта:

In [51]:
!ls ../../data/msmarco-sample

msmarco-docdev-qrels.tsv.gz    msmarco-doctrain-qrels.tsv.gz
msmarco-docdev-queries.tsv.gz  msmarco-doctrain-queries.tsv.gz
msmarco-docs.tsv


Формат этого сэмпла идентичен формату датасета VK MARCO, который будет использоваться во всех последующих ДЗ (в т.ч. в финальном проекте), поэтому весь код этого примера вы сможете переиспользовать в ДЗ.

Рассмотрим подробнее формат датасета, нас тут в первую очередь интересуют 3 файла:
- msmarco-docs.tsv              - в нем лежат 200 документов, по которым будем искать, в формате DOC_ID\tURL\tTITLE\tBODY
- msmarco-docdev-queries.tsv.gz - тут лежат 100 запросов, по которым мы хотим искать, в формате QUERY_ID\tQUERY
- msmarco-docdev-qrels.tsv.gz   - тут хранятся оценки для пар запрос-документ, в формате QUERY_ID 0 DOC_ID LABEL

Тут:
- \t       - это табуляция
- DOC_ID   - это уникальный идентификатор документа, он имеет вид строки типа "D2749594"
- QUERY_ID - это уникальный идентификатор запроса, это просто число, например 42568
- LABEL    - это оценка релевантности, в датасете MS MARCO используется бинарный критерий релевантности и поэтому она может принимать значения только 0 (документ нерелевантен запросу) или 1 (документ релевантен запросу).

Важный момент: в msmarco-docdev-qrels.tsv.gz хранятся только релевантные пары запрос-документ, т.е. LABEL всегда равен 1! Все остальные пары запрос-документ считаем нерелевантными!

Документы состоят из 2х зон: TITLE и BODY

Теперь попробуем загрузить наш датасет.

## Загружаем датасет MS MARCO

Импортируем все что понадобится для дальнейшей работы:

In [52]:
import math
import pathlib
import shutil

import pandas as pd

from whoosh import analysis
from whoosh import fields
from whoosh import index
from whoosh.lang import porter
from whoosh.lang.snowball import english, russian
from whoosh import qparser

### Загружаем запросы

In [53]:
# Путь до корня проекта
project_root_dir = pathlib.Path("../..")

# Путь до датасета
data_dir = project_root_dir.joinpath("data/msmarco-sample")

# Файл с запросами
queries_file = data_dir.joinpath(f"msmarco-docdev-queries.tsv.gz")

# Загружаем запросы во фрейм
queries_df = pd.read_csv(queries_file, sep='\t', header=None)
queries_df.columns = ['query_id', 'query_text']
print(queries_df.head(5))

   query_id                                   query_text
0    869891  what kind of party is the cooperative party
1    488676                               retinue define
2    595568                               what cisco ios
3   1039361              who is the author of soccerland
4   1089511                        tooth veneers process


### Загружаем оценки

In [54]:
# Файл с оценками
qrels_file = data_dir.joinpath(f"msmarco-docdev-qrels.tsv.gz")

# Загружаем оценки во фрейм
qrels_df = pd.read_csv(qrels_file, sep=' ', header=None)
qrels_df.columns = ['query_id', 'unused', 'doc_id', 'label']
print(qrels_df.head(5))

   query_id  unused    doc_id  label
0     42568       0  D2749594      1
1     53813       0   D779848      1
2     54843       0  D1475635      1
3     60357       0   D740627      1
4     61180       0   D971677      1


### Конвертируем запросы и оценки в более удобный для дальнейшей работы формат

In [55]:
# Представляем запросы в виде словаря: Query ID -> Text
query_id2text = {query_id: query_text for query_id, query_text in zip(queries_df['query_id'], queries_df['query_text'])}
print(query_id2text)

{869891: 'what kind of party is the cooperative party', 488676: 'retinue define', 595568: 'what cisco ios', 1039361: 'who is the author of soccerland', 1089511: 'tooth veneers process', 302337: 'how much are servers pay at olive garden', 605651: 'what county is emmett ks in', 780336: 'what is orthorexia', 602352: 'what county is alton bay,nh in?', 660479: 'what food helps to produce collagen', 1086765: 'what are the tax benefits of a heloc', 733892: 'what is considered inpatient types', 1004228: 'when is champaign il midterm elections', 1051339: 'what is medical aki', 241246: 'how long can a hippo hold its breath underwater', 42568: 'average salary for public relations manager', 250228: 'how long does a stress test usually take', 1094081: 'is minneapolis sick and safe time law', 1084889: 'what dna molecules bond with each other', 910818: 'what type of cancer did jim kelly have', 504335: 'stress effects on the body', 667136: 'what happens when catalytic converter is bad', 455659: 'money

In [56]:
# Представляем оценки в виде словаря: Query ID -> Doc ID -> Label (relevance)
qrels = {}
for i in range(0, len(qrels_df)):
    qrels_row = qrels_df.iloc[i]
    query_id = qrels_row['query_id']
    doc_id = qrels_row['doc_id']
    label = qrels_row['label']
    if label != 1:
        raise Exception(f"invalid label in qrels: doc_id = {doc_id}")

    doc_id2label = qrels.get(query_id)
    if doc_id2label is None:
        doc_id2label = {}
        qrels[query_id] = doc_id2label
        doc_id2label[doc_id] = label
print(qrels)

{42568: {'D2749594': 1}, 53813: {'D779848': 1}, 54843: {'D1475635': 1}, 60357: {'D740627': 1}, 61180: {'D971677': 1}, 65584: {'D1674076': 1}, 102506: {'D612380': 1}, 137440: {'D2392325': 1}, 138223: {'D1288403': 1}, 142382: {'D82515': 1}, 148564: {'D2557627': 1}, 162662: {'D56193': 1}, 241246: {'D2638732': 1}, 250228: {'D446341': 1}, 250636: {'D521463': 1}, 282214: {'D1683151': 1}, 293401: {'D128007': 1}, 302337: {'D1212479': 1}, 307504: {'D3543950': 1}, 389385: {'D1471949': 1}, 409694: {'D2405355': 1}, 413079: {'D899910': 1}, 414799: {'D209747': 1}, 416846: {'D1477895': 1}, 417362: {'D17357': 1}, 420400: {'D551076': 1}, 436847: {'D2064661': 1}, 455659: {'D1348364': 1}, 488676: {'D1498313': 1}, 504335: {'D1028631': 1}, 509111: {'D357179': 1}, 515335: {'D1269311': 1}, 525467: {'D2458804': 1}, 539601: {'D1295431': 1}, 558548: {'D2526315': 1}, 560245: {'D60086': 1}, 573157: {'D1647749': 1}, 573452: {'D1811785': 1}, 582848: {'D590557': 1}, 583798: {'D2202662': 1}, 595568: {'D1903511': 1}, 

### Загружаем документы

In [57]:
# Функция которая читает все документы в один большой список
def read_docs(docs_file):
    docs = []
    with open(docs_file, 'rt') as f:
        for line in f:
            # Парсим следующую строку
            parts = line.rstrip('\n').split('\t')
            if len(parts) != 4:
                logging.warning("invalid line: num_lines = %d num_parts = %d", self.num_lines, len(parts))
            doc_id, url, title, body = parts

            # Валидируем
            if not doc_id or len(doc_id) < 2:
                raise RuntimeError(f"invalid doc id: num_lines = {self.num_lines}")
            if not url:
                raise RuntimeError(f"invalid url: num_lines = {self.num_lines}")
           
            # Заголовок вида '.' обозначает пустой заголовок (особенность датасета MS MARCO)
            if title == '.':
                title = ''

            # Пакуем данные документа в словарь
            doc = {'url': url, 'title': title, 'body': body, 'docid': doc_id}
            docs.append(doc)
    return docs

# Файл с документами
docs_file = data_dir.joinpath("msmarco-docs.tsv")

# Загружаем все документы
docs = read_docs(docs_file)
print(f"Loaded {len(docs)} docs")

Loaded 200 docs


## Индексируем документы

### Готовимся к индексации

In [58]:
# Временная папка для индекса
index_dir = pathlib.Path("/tmp/index")

# Удаляем старый индекс, если такой существует
shutil.rmtree(index_dir, ignore_errors=True)

# Создаем заново папку под индекс
index_dir.mkdir(exist_ok=True, parents=True)

# Создаем analyzer (без поддержки стемминга) который будет использоваться для обработки текстов запроса и документа
#analyzer = analysis.StandardAnalyzer()

# Создаем Analyzer с поддержкой стемминга для английского языка
stemmer = english.EnglishStemmer()
stemfn = stemmer.stem
analyzer = analysis.StemmingAnalyzer(stemfn=stemfn)

# Создаем схему индекса: будем хранить Doc ID, URL, и тексты TITLE и BODY
schema = fields.Schema(
    doc_id=fields.ID(stored=True), 
    url=fields.TEXT(stored=True),
    title=fields.TEXT(analyzer=analyzer),
    body=fields.TEXT(analyzer=analyzer)
)

### Создаем и заполняем индекс

In [59]:
# Создаем индекс согласно объявленной схеме
ix = index.create_in(index_dir, schema)

# Объект-writer который будет использоваться для добавления новых документов в индекс
writer = ix.writer()

# Добавляем все наши документы в индекс
for doc in docs:
    writer.add_document(doc_id=doc['docid'], url=doc['url'], title=doc['title'], body=doc['body'])

# Записываем на диск
writer.commit()

Посмотрим на структуру индекса:

In [60]:
!ls /tmp/index

_MAIN_1.toc  MAIN_w704nz5ppuoxbjmq.seg	MAIN_WRITELOCK


## Ищем запросы в индексе

In [61]:
# Функция для подсчета DCG@K, понадобится нам для расчета метрик
def dcg(y, k=10):
    """Computes DCG@k for a single query.
            
    y is a list of relevance grades sorted by position.
    len(y) could be <= k.
    """     
    r = 0.
    for i, y_i in enumerate(y):
        p = i + 1 # position starts from 1
        r += (2 ** y_i - 1) / math.log(1 + p, 2)
        if p == k:
            break
    return r

# Готовим парсер запросов. Будем искать сразу в 2х полях (TITLE и BODY) используя т.н. кворум (булев поиск с "мягким И")
qp = qparser.MultifieldParser(['title', 'body'], schema=ix.schema, group=qparser.OrGroup.factory(0.9))
qp.remove_plugin_class(qparser.WildcardPlugin) # Ускоряет поиск

# Суммарный DCG@10 по всем запросам
dcg10_sum = 0

# Вспомогательная функция, которая превращает None в 0
def none_to_label(label):
    return 0 if label is None else label
    
# Создаем объект searcher с помощью которого будем искать в индексе
with ix.searcher() as searcher:
    # Ищем каждый запрос по очереди
    query_ids = sorted(query_id2text.keys())
    for query_id in query_ids:
        query_text = query_id2text[query_id]
        
        # Парсим запрос
        query = qp.parse(query_text)

        # Собственно сам поиск
        results = searcher.search(query)

        # Достаем из результатов поиска т.н. "хиты" -- найденные документы.
        # Результаты уже отранжированы с помощью формулы BM25!
        num_hits = results.scored_length()
        hits = [results[i] for i in range(num_hits)]
        #print(hits)

        # Найденные Doc ID
        found_doc_ids = [hit['doc_id'] for hit in hits]

        # Получаем все известные оценки для этого запроса
        doc_id2label = qrels[query_id]

        # Формируем сортированный по убыванию ранка (BM25) список оценок для всех найденных документов
        labels = [none_to_label(doc_id2label.get(doc_id)) for doc_id in found_doc_ids]

        # Считаем DCG@10 для этого запроса
        dcg10 = dcg(labels, k=10)
        dcg10_sum += dcg10
        print(f"Next query: query_id = {query_id} query_text = '{query_text}' dcg@10 = {dcg10:.3f}")

Next query: query_id = 42568 query_text = 'average salary for public relations manager' dcg@10 = 1.000
Next query: query_id = 53813 query_text = 'binding spell definition' dcg@10 = 1.000
Next query: query_id = 54843 query_text = 'blueberries pint' dcg@10 = 1.000
Next query: query_id = 60357 query_text = 'calories in ham sandwich' dcg@10 = 1.000
Next query: query_id = 61180 query_text = 'calories italian beef sandwich' dcg@10 = 1.000
Next query: query_id = 65584 query_text = 'can chewing gum prevent heartburn' dcg@10 = 1.000
Next query: query_id = 102506 query_text = 'cost of appendectomy surgery in usa' dcg@10 = 1.000
Next query: query_id = 137440 query_text = 'definition of prostration' dcg@10 = 1.000
Next query: query_id = 138223 query_text = 'definition of slouch' dcg@10 = 1.000
Next query: query_id = 142382 query_text = 'determining marginal cost from total cost equation' dcg@10 = 1.000
Next query: query_id = 148564 query_text = 'difference between router and firewall' dcg@10 = 1.0

### Считаем средние метрики

В случае бинарной релевантности метрика DCG является не очень подходящей для оценки качества поиска, но мы тем не менее будем использовать именно ее т.к. в ДЗ будет использоваться датасет VK MARCO в котором label уже представляет из себя асессорскую оценку в диапазоне [0,4]


In [62]:
# Полное число запросов
num_queries = len(query_id2text)

# Среднее DCG@10
dcg10_avg = dcg10_sum / num_queries
print(dcg10_avg)

0.9439278926071435
